# 2) Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

# 3) Importing dataframes

In [2]:
# Creating a path variable
path = r'C:\Users\Aaron\CareerFoundry\Instacart Basket Analysis'

In [3]:
# Importing the customers data set
df_cust = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)

In [4]:
# Checking the dataframe
df_cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


# 4) Wrangling and Cleaning up customers dataframe

### Dropping Gender, data_joined, and fam_status columns

In [5]:
df_cust = df_cust.drop(columns = ['Gender', 'date_joined', 'fam_status'])

In [6]:
# Checking our work
df_cust.head()

,user_id,First Name,Surnam,STATE,Age,n_dependants,income
0,26711,Deborah,Esquivel,Missouri,48,3,165665
1,33890,Patricia,Hart,New Mexico,36,0,59285
2,65803,Kenneth,Farley,Idaho,35,2,99568
3,125935,Michelle,Hicks,Iowa,40,0,42049
4,130797,Ann,Gilmore,Maryland,26,1,40374


### Renaming some of the columns

In [7]:
df_cust.rename(columns = {'First Name': 'first_name', 'Surnam': 'last_name', 'STATE': 'state', 'Age': 'age'}, inplace = True)

In [8]:
# Checking our work
df_cust.head()

,user_id,first_name,last_name,state,age,n_dependants,income
0,26711,Deborah,Esquivel,Missouri,48,3,165665
1,33890,Patricia,Hart,New Mexico,36,0,59285
2,65803,Kenneth,Farley,Idaho,35,2,99568
3,125935,Michelle,Hicks,Iowa,40,0,42049
4,130797,Ann,Gilmore,Maryland,26,1,40374


### Learning more about our dataframe

In [9]:
df_cust.shape

(206209, 7)

In [10]:
df_cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   first_name    194950 non-null  object
 2   last_name     206209 non-null  object
 3   state         206209 non-null  object
 4   age           206209 non-null  int64 
 5   n_dependants  206209 non-null  int64 
 6   income        206209 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 11.0+ MB


### Changing user_id data type to float64
(same datatype as our ords_prods_merge user_id)

In [11]:
df_cust['user_id'] = df_cust['user_id'].astype('float64')

In [12]:
# Checking our work
df_cust['user_id'].dtype

dtype('float64')

### Describing dataframe

In [13]:
df_cust.describe()

,user_id,age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


#### All values look relatively normal. Only value that may be questionable is the max income but even though it's high, it's still possible; so therefore it's not that unlikely to happen.

# 5) Data Consistency Checks

## Looking for missing data

In [14]:
df_cust.isnull().sum()

user_id             0
first_name      11259
last_name           0
state               0
age                 0
n_dependants        0
income              0
dtype: int64

#### Since the missing values make up just about 5% of the entire dataframe, we can remove them from the dataframe.

In [15]:
df_cust_clean = df_cust[df_cust['first_name'].isnull() == False]

In [16]:
# Checking to see our new dataframe's shape
df_cust_clean.shape

(194950, 7)

In [17]:
# Checking to see if we have any missing values left
df_cust_clean.isnull().sum()

user_id         0
first_name      0
last_name       0
state           0
age             0
n_dependants    0
income          0
dtype: int64

## Looking for duplicate data

In [18]:
df_cust_dup = df_cust_clean[df_cust_clean.duplicated()]

In [19]:
# Checking the duplicated values
df_cust_dup.head()

,user_id,first_name,last_name,state,age,n_dependants,income


#### No duplicated values! Nice!

## Checking for mixed-type data

In [20]:
# Copy and Pasted code from above Exercise 4.5, changed it to df_cust_clean
for col in df_cust_clean.columns.tolist():
    weird = (df_cust_clean[[col]].applymap(type) != df_cust_clean[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_cust_clean[weird]) > 0:
        print(col)

#### No column was printed in the output. This means no columns contain mixed-type data.

#### Just to make sure, I'll do the function again. However, this time I will make function == 0. This should provide an output of all columns that are NOT mixed-type data.

In [21]:
for col in df_cust_clean.columns.tolist():
    weird = (df_cust_clean[[col]].applymap(type) != df_cust_clean[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_cust_clean[weird]) == 0:
        print(col)

user_id
first_name
last_name
state
age
n_dependants
income


#### Since it printed all of our columns, this means they don't contain mixed-type data.

# Importing ords_prods_merge

In [22]:
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge_TASK49UPDATE.pkl'))

In [24]:
# Check the df
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_time,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price_spent,spending_flag,median_since_prior_orders,order_frequency_flag
0,2539329.0,1.0,1.0,2.0,8.0,11.0,196,1.0,0.0,both,Soda,77.0,7.0,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,10.0,New customer,6.367797,Low spender,20.0,Regular customer
1,2398795.0,1.0,2.0,3.0,7.0,15.0,196,1.0,1.0,both,Soda,77.0,7.0,9.0,Mid-range product,Regularly busy,Slowest days,Average orders,10.0,New customer,6.367797,Low spender,20.0,Regular customer
2,473747.0,1.0,3.0,3.0,12.0,21.0,196,1.0,1.0,both,Soda,77.0,7.0,9.0,Mid-range product,Regularly busy,Slowest days,Most orders,10.0,New customer,6.367797,Low spender,20.0,Regular customer
3,2254736.0,1.0,4.0,4.0,7.0,29.0,196,1.0,1.0,both,Soda,77.0,7.0,9.0,Mid-range product,Least busy,Slowest days,Average orders,10.0,New customer,6.367797,Low spender,20.0,Regular customer
4,431534.0,1.0,5.0,4.0,15.0,28.0,196,1.0,1.0,both,Soda,77.0,7.0,9.0,Mid-range product,Least busy,Slowest days,Most orders,10.0,New customer,6.367797,Low spender,20.0,Regular customer


In [23]:
# Original display didn't show us all of the columns. Wanted to change it to show all columns.
pd.options.display.max_columns = None

#### Deleting the _merge and busiest_day columns for ords_prods_merge

In [25]:
ords_prods_merge = ords_prods_merge.drop(columns = ['_merge', 'busiest_day'])

In [26]:
# Checking ords_prods_merge data type
ords_prods_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32435070 entries, 0 to 32435069
Data columns (total 22 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   order_id                   float64
 1   user_id                    float64
 2   order_number               float64
 3   orders_day_of_week         float64
 4   order_time                 float64
 5   days_since_prior_order     float64
 6   product_id                 int64  
 7   add_to_cart_order          float64
 8   reordered                  float64
 9   product_name               object 
 10  aisle_id                   float64
 11  department_id              float64
 12  prices                     float64
 13  price_range_loc            object 
 14  busiest_days               object 
 15  busiest_period_of_day      object 
 16  max_order                  float64
 17  loyalty_flag               object 
 18  avg_price_spent            float64
 19  spending_flag              object 
 20  

### Checking df_cust_clean again

In [27]:
df_cust_clean.head()

,user_id,first_name,last_name,state,age,n_dependants,income
0,26711.0,Deborah,Esquivel,Missouri,48,3,165665
1,33890.0,Patricia,Hart,New Mexico,36,0,59285
2,65803.0,Kenneth,Farley,Idaho,35,2,99568
3,125935.0,Michelle,Hicks,Iowa,40,0,42049
4,130797.0,Ann,Gilmore,Maryland,26,1,40374


#### We can see that both Customer data and ords_prods_merge have a "user_id" column. We can use this as their key column that allows us to combine them.

# Merging customer data with ords_prods_merge

In [28]:
ords_prods_cust = ords_prods_merge.merge(df_cust_clean, on = 'user_id', indicator = True)

In [29]:
# Checking the new dataframe
ords_prods_cust.head()

,order_id,user_id,order_number,orders_day_of_week,order_time,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price_spent,spending_flag,median_since_prior_orders,order_frequency_flag,first_name,last_name,state,age,n_dependants,income,_merge
0,2539329.0,1.0,1.0,2.0,8.0,11.0,196,1.0,0.0,Soda,77.0,7.0,9.0,Mid-range product,Regularly busy,Average orders,10.0,New customer,6.367797,Low spender,20.0,Regular customer,Linda,Nguyen,Alabama,31,3,40423,both
1,2398795.0,1.0,2.0,3.0,7.0,15.0,196,1.0,1.0,Soda,77.0,7.0,9.0,Mid-range product,Slowest days,Average orders,10.0,New customer,6.367797,Low spender,20.0,Regular customer,Linda,Nguyen,Alabama,31,3,40423,both
2,473747.0,1.0,3.0,3.0,12.0,21.0,196,1.0,1.0,Soda,77.0,7.0,9.0,Mid-range product,Slowest days,Most orders,10.0,New customer,6.367797,Low spender,20.0,Regular customer,Linda,Nguyen,Alabama,31,3,40423,both
3,2254736.0,1.0,4.0,4.0,7.0,29.0,196,1.0,1.0,Soda,77.0,7.0,9.0,Mid-range product,Slowest days,Average orders,10.0,New customer,6.367797,Low spender,20.0,Regular customer,Linda,Nguyen,Alabama,31,3,40423,both
4,431534.0,1.0,5.0,4.0,15.0,28.0,196,1.0,1.0,Soda,77.0,7.0,9.0,Mid-range product,Slowest days,Most orders,10.0,New customer,6.367797,Low spender,20.0,Regular customer,Linda,Nguyen,Alabama,31,3,40423,both


In [30]:
# Checking ords_prods_cust shape
ords_prods_cust.shape

(30658154, 29)

In [31]:
# Checking info
ords_prods_cust.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30658154 entries, 0 to 30658153
Data columns (total 29 columns):
 #   Column                     Dtype   
---  ------                     -----   
 0   order_id                   float64 
 1   user_id                    float64 
 2   order_number               float64 
 3   orders_day_of_week         float64 
 4   order_time                 float64 
 5   days_since_prior_order     float64 
 6   product_id                 int64   
 7   add_to_cart_order          float64 
 8   reordered                  float64 
 9   product_name               object  
 10  aisle_id                   float64 
 11  department_id              float64 
 12  prices                     float64 
 13  price_range_loc            object  
 14  busiest_days               object  
 15  busiest_period_of_day      object  
 16  max_order                  float64 
 17  loyalty_flag               object  
 18  avg_price_spent            float64 
 19  spending_flag      

In [32]:
# Describing ords_prods_cust
ords_prods_cust.describe()

,order_id,user_id,order_number,orders_day_of_week,order_time,days_since_prior_order,product_id,add_to_cart_order,reordered,aisle_id,department_id,prices,max_order,avg_price_spent,median_since_prior_orders,age,n_dependants,income
count,3.065815e+07,3.065815e+07,3.065815e+07,3.065815e+07,3.065815e+07,3.065815e+07,3.065815e+07,3.065815e+07,3.065815e+07,3.062974e+07,3.062974e+07,3.062496e+07,3.065815e+07,3.065815e+07,3.065815e+07,3.065815e+07,3.065815e+07,3.065815e+07
mean,1.710743e+06,1.029799e+05,1.712357e+01,2.738023e+00,1.342507e+01,1.110348e+01,2.557962e+04,8.353481e+00,5.896156e-01,7.119861e+01,9.918011e+00,7.791916e+00,3.301238e+01,1.191497e+01,1.002738e+01,4.947327e+01,1.501695e+00,9.945713e+04
std,9.872965e+05,5.941782e+04,1.751515e+01,2.089944e+00,4.244493e+00,8.494075e+00,1.409647e+04,7.127682e+00,4.919035e-01,3.821150e+01,6.281766e+00,4.242307e+00,2.512152e+01,8.262590e+01,6.322970e+00,1.847860e+01,1.118471e+00,4.306414e+04
min,2.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00,1.000000e+00,0.000000e+00,1.800000e+01,0.000000e+00,2.590300e+04
25%,8.559500e+05,5.156600e+04,5.000000e+00,1.000000e+00,1.000000e+01,5.000000e+00,1.353500e+04,3.000000e+00,0.000000e+00,3.100000e+01,4.000000e+00,4.200000e+00,1.300000e+01,7.388276e+00,6.000000e+00,3.300000e+01,1.000000e+00,6.706600e+04
50%,1.711162e+06,1.025990e+05,1.100000e+01,3.000000e+00,1.300000e+01,8.000000e+00,2.526500e+04,6.000000e+00,1.000000e+00,8.300000e+01,9.000000e+00,7.400000e+00,2.600000e+01,7.825316e+00,8.000000e+00,4.900000e+01,2.000000e+00,9.661000e+04
75%,2.565298e+06,1.543850e+05,2.400000e+01,5.000000e+00,1.600000e+01,1.400000e+01,3.794000e+04,1.100000e+01,1.000000e+00,1.070000e+02,1.600000e+01,1.130000e+01,4.700000e+01,8.253789e+00,1.200000e+01,6.500000e+01,3.000000e+00,1.279300e+05
max,3.421082e+06,2.062090e+05,9.900000e+01,6.000000e+00,2.300000e+01,3.000000e+01,4.968800e+04,1.450000e+02,1.000000e+00,1.340000e+02,2.100000e+01,2.500000e+01,9.900000e+01,2.500542e+04,3.000000e+01,8.100000e+01,3.000000e+00,5.939010e+05


# Exporting ords_prods_cust

In [33]:
ords_prods_cust.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_cust.pkl'))